# Model Execution

In this notebook we will:

- import the data from the provided sql database using the data processing functions in helper_functions.py
- create the master dataset
- establish a baseline model
- tune parameters to create a final model

We will start by importing the required packages and defining the sql connection

In [1]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

import helper_functions

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer

In [2]:
con = sqlite3.connect("synthetic_claims.db")

Next we will define the date ranges for the features and the label data and then import the label data and demographics features. Since the prompt asked to predict ER visits in the next 6 months, we will use the final 6 months of the data from the utilization table to create the label data. This time frame is August 2019 to January 2020. The feature end date is thus July 2019. Feature start date will be defined dynamically later to allow for features over multiple time frames. In production, I would define these values dynamically based on todays date, so something like:

- as_of_end_date = today()
- as_of_date = as_of_end_date - 180 days
- feature_end_date = as_of_date - 1 day

In [3]:
as_of_date_str = '2019-08-01'
as_of_date = datetime.strptime(as_of_date_str, '%Y-%m-%d').date()
as_of_end_date = as_of_date + relativedelta(months=6)

feature_end_date = as_of_date - relativedelta(months=1)

In [4]:
label_data = helper_functions.load_label_data(as_of_date, as_of_end_date, con)

# note that the demographics features, gender and birth year, are static, so we dont need the 
# feature start and end dates. We do need the as of date to establish that the person is alive though
demographics_features = helper_functions.load_demographics_features(as_of_date, con)

Next we will load the rest of the features and combine everything into a master dataframe. A preview of this dataframe is below. I did consider creating a join_feature_dfs helper function that accepted the master_df after joining the labels and the demographics info, the month_val list and a list of the 3 feature dfs within the for loop. I decided against it as it made the notebook less intuitive to read. I would probably do that in production though to avoid the repetitive code below.

In [5]:
month_vals = [3,12]

master_df = pd.merge(label_data,demographics_features, on = 'patient_id',how = 'left')

for i in month_vals:
    print(i)
    
    feature_start_date = feature_end_date - relativedelta(months=i)
    
    procedure_features = helper_functions.load_procedure_features(feature_start_date,feature_end_date, con)
    diagnosis_features = helper_functions.load_diagnosis_features(feature_start_date,feature_end_date, con)
    utilization_features = helper_functions.load_utilization_features(feature_start_date, feature_end_date, con)
    
    #rename appropriate features to specify the month value used to generate them
    procedure_features = helper_functions.rename_feature_dfs(procedure_features,i)
    diagnosis_features = helper_functions.rename_feature_dfs(diagnosis_features,i)
    utilization_features = helper_functions.rename_feature_dfs(utilization_features,i)
    
    master_df = pd.merge(master_df,procedure_features, on = 'patient_id',how = 'left')
    master_df = pd.merge(master_df,diagnosis_features, on = 'patient_id',how = 'left')
    master_df = pd.merge(master_df,utilization_features, on = 'patient_id',how = 'left')

3
12


In [6]:
master_df.head()

,patient_id,er_visit,female_bool,birth_year,3_month_endocrine_ops,3_month_misc_proc,3_month_ear_proc,3_month_hemic_lymphatic_proc,3_month_respiratory_proc,3_month_cardiovascular_proc,...,12_month_mental_dia,12_month_inj_poison_dia,12_month_pregnancy_puerperium_dia,12_month_ill_defined_dia,12_month_respiratory_dia,12_month_circulatory_dia,12_month_genitourinary_dia,12_month_off_visits_util,12_month_er_visits_util,12_month_admits_util
0,000a708f45d7cea9,0,1,1944,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,11.0,0.0,10.0,0.0,1.0,0.0,0.0
1,000dce3d67c354f1,0,1,1952,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0019ec94bfe6e6c7,1,1,1948,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,2.0,3.0,0.0,4.0,0.0,0.0
3,001aa34ee5a87397,0,0,1944,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0022707b432b96ca,1,0,1945,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,1.0,11.0,1.0,7.0,0.0,0.0


Based on the below, since ER visits are relatively rare, we may consider weighting the label data in order to boost the precision of the model. We can also close the sql connection as we are done with it.

In [7]:
sql = """
SELECT strftime('%Y', DATE) AS Year, 
       strftime('%m', DATE) AS Month, 
       SUM(ER_Visits) AS ER_Visits, 
       COUNT(ER_Visits) AS Row_Count,
       SUM(ER_Visits) * 1.0 / COUNT(ER_Visits) AS Monthly_Avg_ER_Visits_per_Patient
FROM Utilization
GROUP BY strftime('%Y', DATE), strftime('%m', DATE)
"""

pd.read_sql_query(sql, con)

,Year,Month,ER_Visits,Row_Count,Monthly_Avg_ER_Visits_per_Patient
0,2018,02,123,4831,0.025461
1,2018,03,132,4865,0.027133
2,2018,04,154,4901,0.031422
3,2018,05,137,4931,0.027783
4,2018,06,175,4967,0.035233
5,2018,07,165,5001,0.032993
6,2018,08,165,5046,0.032699
7,2018,09,174,5086,0.034212
8,2018,10,146,5125,0.028488
9,2018,11,166,5166,0.032133


In [8]:
con.close()

Next we will split out our dataset and run a vanilla gradient boosting model, both with and without weighting, to establish a baseline model before getting into parameter tuning.

In [9]:
X = master_df.loc[:,(master_df.columns != 'patient_id') & (master_df.columns != 'er_visit')]
y = master_df['er_visit']
#not all alive patients have data in the other tables. Im assuming this means its appropriate to fill with 0s
X = X.fillna(0) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

In [10]:
clf = GradientBoostingClassifier()

In [11]:
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
print('accuracy: ', (tp + tn) / (tn + fp + fn + tp))
print('precision: ', tp / (tp + fp))

accuracy:  0.8718354430379747
precision:  0.43333333333333335


In [12]:
weights = compute_sample_weight(class_weight='balanced', y=y_train)
clf.fit(X_train, y_train, sample_weight = weights)
y_hat = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
print('accuracy: ', (tp + tn) / (tn + fp + fn + tp))
print('precision: ', tp / (tp + fp))

accuracy:  0.7175632911392406
precision:  0.22128851540616246


In [13]:
print(max(weights) / min(weights))

6.176136363636363


Interestingly, the weighted model appears to be overfitting for the positive labels. The above division means that the patients are 6x more likely to not go to the ER than to in the span of 6 months. The actual values of the weights are unimportant, the ratio between the two is what impacts the model. Just out of curiousity, lets try some smaller weight ratios just to see how it affects the model.

In [14]:
for w in range(2,6):
    
    custom_weight = y_train.copy()
    
    custom_weight[y_train == 1] = w
    custom_weight[y_train == 0] = 1
    
    clf.fit(X_train, y_train, sample_weight = custom_weight)
    y_hat = clf.predict(X_test)
        
    tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
    print('for weight ' + str(w))
    print('accuracy: ', (tp + tn) / (tn + fp + fn + tp))
    print('precision: ',tp / (tp + fp))
    print('')

for weight 2
accuracy:  0.8694620253164557
precision:  0.45569620253164556

for weight 3
accuracy:  0.8401898734177216
precision:  0.3472222222222222

for weight 4
accuracy:  0.8085443037974683
precision:  0.29207920792079206

for weight 5
accuracy:  0.7729430379746836
precision:  0.26199261992619927



While the 2 to 1 weight ratio is competitive with the unweighted model, it appears we are barking up the wrong tree so we will use the unweighted gradient boosting model as a baseline. Next we will utilize grid search cross validation to iterate through a variety of parameters to evaluate the effect on model performance. 

In [21]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score)}

parameters = {
    "learning_rate": [0.05,.1,.15,.2],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "squared_error"],
    "subsample":[0.5, 1.0],
    "n_estimators":[100,250]
    }

clf = GridSearchCV(GradientBoostingClassifier(random_state = 6), 
                   parameters,scoring=scoring,refit=False,cv=2, n_jobs=-1)
clf.fit(X_train, y_train)
cv_results_df=pd.DataFrame.from_dict(clf.cv_results_)

# since cv=2, there two splits, split0 and split1
cv_results_df['total_precision'] = (cv_results_df['split0_test_precision'] + cv_results_df['split1_test_precision']) / 2

The below is the highest precision combination of parameters from the cross validation. We will then utilize these parameters to create a final model and compare it to the baseline.

In [22]:
cv_results_df.loc[cv_results_df['total_precision'].idxmax()]['params']

{'criterion': 'friedman_mse',
 'learning_rate': 0.05,
 'max_features': 'sqrt',
 'n_estimators': 100,
 'subsample': 0.5}

In [23]:
clf =GradientBoostingClassifier(criterion='friedman_mse',
                                learning_rate=0.05,
                                max_features = 'sqrt',
                                n_estimators = 100,
                                subsample = 0.5
)

clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
    
tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
print('final model')
print('accuracy: ', (tp + tn) / (tn + fp + fn + tp))
print('precision: ',tp / (tp + fp))


final model
accuracy:  0.877373417721519
precision:  0.6


Indeed there is a small jump in accuracy and a larger one in precision. [Hooray](https://www.youtube.com/watch?v=ATWGa5JiNtQ)! Finally we will examine the feature importance.

In [18]:
feature_importances = pd.DataFrame(clf.feature_importances_, X_train.columns)
feature_importances = feature_importances.rename(columns={0: 'feature_importance'})
feature_importances = feature_importances.sort_values(by = 'feature_importance', ascending = False)

In [19]:
feature_importances.head(20)

,feature_importance
12_month_er_visits_util,0.107572
12_month_misc_proc,0.081659
12_month_mental_dia,0.064232
12_month_respiratory_dia,0.055059
birth_year,0.043818
12_month_endocrine_metabolic_immuno_dia,0.043165
12_month_circulatory_dia,0.029613
12_month_off_visits_util,0.028146
12_month_musculoskeletal_connective_dia,0.027574
12_month_other_icd_dia,0.024274


In [20]:
feature_importances.tail()

,feature_importance
3_month_female_proc,0.0
3_month_obstetrical_proc,0.0
12_month_obstetrical_proc,0.0
3_month_ear_proc,0.0
3_month_pregnancy_puerperium_dia,0.0


The fact that the top features are dominated by 12 month calculations, it would be appropriate to iterate on even larger timelines, data permitting. Additionally there are some 0 importance features that would be appropriate to remove. 

That wraps the model execution! I hope you found it enlightening.

Daniel